In [69]:
import numpy as np
import pandas as pd
print("A")

A


In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
transactions = pd.read_csv("/content/drive/My Drive/transactions.csv")
fraudsters = pd.read_csv("/content/drive/My Drive/fraudsters.csv")

In [72]:
rows = []
transactions["is_fraudulent"] = 0
#for i,row in fraudsters.iterrows():
    #print(row['user_id'].dtype,transactions['USER_ID'].dtype)
    #transactions[ transactions.USER_ID == row["user_id"] ]
for i,row in fraudsters.iterrows():
    transactions.loc[ transactions.USER_ID == row["user_id"], "is_fraudulent" ] = 1

In [73]:
from sklearn.utils import shuffle
df = shuffle(transactions.copy())
columns = list(df.columns)
columns[0] = 'Unnamed_0'
df.columns = columns

In [74]:
df = df.drop(["Unnamed_0","CURRENCY","CREATED_DATE"],axis=1)
data= np.array(df.copy())
untransformed_data = data.copy()
print (data.shape)
data = data.astype(str)
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for i in range(data.shape[1] -1):
    data[:,i] = le.fit_transform(data[:,i])
X = data[:,:-1]
y = data[:,-1]
print(X.shape)
print(y.shape)

(688651, 10)
(688651, 9)
(688651,)


In [75]:
Xtrain, Xtest, Xvalidate  = np.split(X, [int(.7*len(X)), int(.8*len(X))])

ytrain, ytest, yvalidate  = np.split(y, [int(.7*len(y)), int(.8*len(y))])

In [76]:
print(Xtrain.shape)
print(Xvalidate.shape)
print(Xtest.shape)
print()
print(ytrain.shape)
print(yvalidate.shape)
print(ytest.shape)
print('\n', X.shape)

Xtest_start_idx = Xtrain.shape[0] 
Xtest_end_idx = Xtest_start_idx + Xtest.shape[0] - 1
print(Xtest_start_idx)
print(Xtest_end_idx)
Xtest_end_idx - Xtest_start_idx

(482055, 9)
(137731, 9)
(68865, 9)

(482055,)
(137731,)
(68865,)

 (688651, 9)
482055
550919


68864

In [77]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(Xtrain, ytrain)

In [78]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    s=[]
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print ("def tree({}):".format(", ".join(feature_names)))
   
    def recurse(node, depth):

        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            s.append(indent + name + str(threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            s.append(indent + name + str(threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, tree_.value[node]))
            s.append(indent + name + str(threshold))
    recurse(0, 1)
    return s
    

In [79]:
'''from sklearn.tree import export_graphviz
export_graphviz(clf, 'tree.dot')
import pydot
(graph,) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')'''

"from sklearn.tree import export_graphviz\nexport_graphviz(clf, 'tree.dot')\nimport pydot\n(graph,) = pydot.graph_from_dot_file('tree.dot')\ngraph.write_png('tree.png')"

In [80]:
res1 = clf.predict(Xtest)

In [81]:
from sklearn.metrics import classification_report
print("DECISION TREE")
print(classification_report(ytest, res1))

DECISION TREE
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     67262
           1       0.79      0.79      0.79      1603

    accuracy                           0.99     68865
   macro avg       0.89      0.89      0.89     68865
weighted avg       0.99      0.99      0.99     68865



In [82]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10,n_jobs=-1,random_state=99)
rf = rf.fit(Xtrain, ytrain)
res = np.array(rf.predict(Xvalidate))

In [83]:
print (Xvalidate.shape)
data[3]
from collections import Counter
c = Counter(res)
c

(137731, 9)


Counter({'0': 136107, '1': 1624})

In [84]:
res_test = rf.predict(Xtest)

In [85]:
from sklearn.metrics import classification_report
print("RANDOM FOREST")
print(classification_report(yvalidate, res))

RANDOM FOREST
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    134561
           1       0.90      0.46      0.61      3170

    accuracy                           0.99    137731
   macro avg       0.94      0.73      0.80    137731
weighted avg       0.99      0.99      0.98    137731



In [86]:
print(classification_report(ytest, res_test))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     67262
           1       0.90      0.44      0.59      1603

    accuracy                           0.99     68865
   macro avg       0.94      0.72      0.79     68865
weighted avg       0.98      0.99      0.98     68865



In [87]:
users = pd.read_csv("/content/drive/My Drive/users.csv")
users.head(3)
user_ids = users["ID"]
Xtest[0] , X[ Xtest_start_idx ]
Xtest[-1] , X[ Xtest_end_idx]

(array(['29960', '2', '13', '187', '1', '4074', '2', '3', '449816'],
       dtype='<U89'),
 array(['29960', '2', '13', '187', '1', '4074', '2', '3', '449816'],
       dtype='<U89'))

In [88]:
i = 0
new_fraudsters = []
old_fraudsters = list(fraudsters["user_id"])
for j in range(Xtest_start_idx,Xtest_end_idx):
    if( (res_test[i] == 1 or res_test[i] == '1') and untransformed_data[j][5] not in old_fraudsters ):
        #print(untransformed_data[j][8],res_test[i])
        new_fraudsters.append( untransformed_data[j][5])
    i+=1

In [89]:
users_ = pd.DataFrame(columns=users.columns)
#Five fraudsters from new list of fraudsters
for i in range(0,10):
    #print(( users[users.ID == new_fraudsters[i]]) )
    users_ = pd.concat([users_,users[users.ID == new_fraudsters[i]]],axis=0 )
users_

,Unnamed: 0,FAILED_SIGN_IN_ATTEMPTS,KYC,BIRTH_YEAR,COUNTRY,STATE,CREATED_DATE,TERMS_VERSION,PHONE_COUNTRY,HAS_EMAIL,ID
2745,2745,0,PASSED,1985,GB,ACTIVE,19:26.7,5/25/2018,GB||JE||IM||GG,1,e56c66d2-b13c-4440-a496-3b25c96e18de
841,841,0,NONE,1990,FR,ACTIVE,31:52.2,NaN,FR,1,cb1ea2ae-8157-4df0-af71-c7107abdd66e
664,664,0,PASSED,1980,GB,ACTIVE,03:47.9,9/20/2018,GB||JE||IM||GG,1,543da93b-03eb-40f3-a6c8-f099d8b1857e
6126,6126,0,PASSED,1995,GB,ACTIVE,22:07.6,1/16/2017,GB||JE||IM||GG,1,88ad610e-2162-46e8-b2be-4bd4f9e753bb
37,37,0,PASSED,1988,GB,ACTIVE,36:38.7,9/20/2018,GB||JE||IM||GG,1,6da7dc35-ab4a-4bb1-9bce-69c4995cac96
7328,7328,0,PASSED,1943,GB,ACTIVE,41:19.9,5/25/2018,GB||JE||IM||GG,1,3c4e712e-06bc-485b-90e1-98ef632bb118
1244,1244,0,PASSED,1994,GB,ACTIVE,30:08.3,9/20/2018,GB||JE||IM||GG,1,fc89a174-cceb-4df2-b04a-4615b01d22cc
8972,9171,0,PASSED,1971,GB,ACTIVE,52:05.6,9/20/2018,GB||JE||IM||GG,1,50552988-e421-4a57-ba08-2f7f9dc6cae4
2516,2516,0,PASSED,1987,GB,ACTIVE,12:26.4,5/25/2018,GB||JE||IM||GG,1,2ef06148-89e7-455c-9757-8ab0be46468b
2995,2995,0,PASSED,1987,BG,ACTIVE,21:53.5,9/20/2018,BG,1,5a78a1dd-4896-49bf-b2cd-8ab0d1572e2b


In [90]:
users_.to_csv("Task2 (new fraudsters).csv")

In [33]:
currency_details = pd.read_csv("/content/drive/My Drive/currency_details.csv")
fx_rates = pd.read_csv("/content/drive/My Drive/fx_rates.csv")

In [34]:
currency_exp = {}
for i,row in currency_details.iterrows():
    currency_exp.update({row["currency"]: row["exponent"]})

In [35]:
base = list(np.unique(fx_rates.base_ccy))
currency_convert = {key:{} for key in base}
for i in base:
    for j,row in fx_rates.iterrows():
        currency_convert[i].update({row["ccy"]:row["rate"]})
        

In [61]:
users_gt = pd.DataFrame(columns = users.columns)
user_id = []
#transaction[ transaction["CURRENCY"] * ]
cur = np.unique( transactions["CURRENCY"])

for i in cur:
  sub_exp = currency_details[ currency_details ["currency"] ==  i]["exponent"].iloc[0]
  #print(sub_exp)
  sub_trans = transactions[transactions["CURRENCY"] == i].copy()
  #display(sub_trans)
  users_u = np.unique(sub_trans["USER_ID"])
  #display(users_u)
  
  for j in users_u:
    co_user = sub_trans[sub_trans.USER_ID == j].head(1)
    #
    if i != "USD":
      rates_usd = fx_rates[fx_rates["base_ccy"] == "USD"].copy()
      rate = rates_usd[rates_usd["ccy"] == i]["rate"]
      #display(rate.iloc[0])
      co_user["AMOUNT"] = co_user["AMOUNT"] * rate.iloc[0]
    co_user["AMOUNT"] = co_user["AMOUNT"] / 10 ** sub_exp
    #display(co_user)
    user_id = co_user.query("AMOUNT>10 and TYPE == 'CARD_PAYMENT' and STATE == 'COMPLETED'")["USER_ID"]
    if(len(user_id)>0):
      user_i = user_id.iloc[0]
    #print(type(users.ID),type(user_i))
    users_gt = pd.concat([users_,users[ users.ID == user_i  ] ],axis=0 )



'''

transaction[ transaction["CURRENCY"] * ]

for i,row in transactions.iterrows():

    currency = row["CURRENCY"]
    factor = currency_exp[currency]
    amount = row["AMOUNT"]
    if (currency != "USD"):
        rate = currency_convert["USD"][currency]
        amount = amount * rate
    current_amount = amount / 10**factor
    if (current_amount > 10):
        if (row["TYPE"] == "CARD_PAYMENT" and row["STATE"] == "COMPLETED" and row["USER_ID"] not in user_id  ):
            users_gt = pd.concat([users_,users[users.ID == row["USER_ID"]  ] ],axis=0 )
            user_id.append(row["USER_ID"])'''
users_gt

,Unnamed: 0,FAILED_SIGN_IN_ATTEMPTS,KYC,BIRTH_YEAR,COUNTRY,STATE,CREATED_DATE,TERMS_VERSION,PHONE_COUNTRY,HAS_EMAIL,ID
870,870,0,PASSED,1994,GB,ACTIVE,07:20.5,3/20/2018,GB||JE||IM||GG,1,f556bacd-b79f-4fdb-9d0c-e7887480c89a


In [67]:
users_gt1 = pd.DataFrame(columns = users.columns)
user_id = []

for i,row in transactions.iterrows():
    currency = row["CURRENCY"]
    factor = currency_exp[currency]
    amount = row["AMOUNT"]
    if(currency != "USD"):
        rate = currency_convert["USD"][currency]
        amount = amount * rate
    current_amount = amount / 10**factor
    if(current_amount > 10):
        if (row["TYPE"] == "CARD_PAYMENT" and row["STATE"] == "COMPLETED" and row["USER_ID"] not in user_id  ):
            users_gt1 = pd.concat([users_gt1, users[users.ID == row["USER_ID"]  ] ],axis=0 )
            user_id.append(row["USER_ID"])
users_gt1

,Unnamed: 0,FAILED_SIGN_IN_ATTEMPTS,KYC,BIRTH_YEAR,COUNTRY,STATE,CREATED_DATE,TERMS_VERSION,PHONE_COUNTRY,HAS_EMAIL,ID
7560,7560,0,PASSED,1963,GB,ACTIVE,24:41.9,9/20/2018,GB||JE||IM||GG,1,7285c1ec-31d0-4022-b311-0ad9227ef7f4
2503,2503,0,PASSED,1977,GB,ACTIVE,51:04.3,9/20/2018,GB||JE||IM||GG,1,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0
7594,7594,0,FAILED,1992,GB,ACTIVE,30:36.5,NaN,GB||JE||IM||GG,1,821014c5-af06-40ff-91f4-77fe7667809f
5897,5897,0,PASSED,1976,GB,ACTIVE,18:52.6,NaN,GB||JE||IM||GG,1,dd1f6199-127f-49ff-ba25-81393b2e66f2
9031,9230,0,PASSED,1988,GB,ACTIVE,38:40.7,5/25/2018,GB||JE||IM||GG,1,20100a1d-12bc-41ed-a5e1-bc46216e9696
...,...,...,...,...,...,...,...,...,...,...,...
3401,3401,0,PASSED,1963,ES,ACTIVE,45:15.3,5/25/2018,ES,1,88986a0b-1512-49e8-a530-4fce1ef2cccf
4073,4073,3,PASSED,1976,FR,ACTIVE,32:49.7,5/25/2018,FR,1,60cc3a7a-62e0-456b-bde9-cd07b9146031
5356,5356,0,PASSED,1991,IE,ACTIVE,48:00.8,9/20/2018,IE,1,b90f3260-6fc7-4583-991e-495bf025e4f3
5077,5077,0,PASSED,1977,GB,ACTIVE,22:54.9,9/20/2018,GB||JE||IM||GG,1,43093946-d738-4020-a844-f1da6406fcfb


In [68]:
users_gt1.to_csv("Task1.csv")

In [38]:
fraudsters_transactions = pd.DataFrame(columns = transactions.columns)

for i,row in users_.iterrows():
    fraudsters_transactions = pd.concat( [fraudsters_transactions, transactions[ transactions.USER_ID == row["ID"] ]],axis=0)


fraudsters_transactions[ fraudsters_transactions.is_fraudulent == '1'  ], fraudsters_transactions[fraudsters_transactions.is_fraudulent == 1]

(Empty DataFrame
 Columns: [Unnamed: 0, CURRENCY, AMOUNT, STATE, CREATED_DATE, MERCHANT_CATEGORY, MERCHANT_COUNTRY, ENTRY_METHOD, USER_ID, TYPE, SOURCE, ID, is_fraudulent]
 Index: [], Empty DataFrame
 Columns: [Unnamed: 0, CURRENCY, AMOUNT, STATE, CREATED_DATE, MERCHANT_CATEGORY, MERCHANT_COUNTRY, ENTRY_METHOD, USER_ID, TYPE, SOURCE, ID, is_fraudulent]
 Index: [])